In [1]:
# Python Version: "3.10.5"
# Java Version: "1.8.0_421"

In [2]:
import subprocess

try:
    # Esegue il comando per ottenere la versione di Java
    version = subprocess.check_output(['java', '-version'], stderr=subprocess.STDOUT)
    print(version.decode('utf-8'))
except FileNotFoundError:
    # Gestisce il caso in cui Java non sia installato
    print("Java non è installato sul sistema. Si prega di installare Java 8 o successiva")
except subprocess.CalledProcessError as e:
    # Gestisce errori legati all'esecuzione del comando
    print(f"Errore durante l'esecuzione del comando Java: {e}")

openjdk version "1.8.0_422-422"
OpenJDK Runtime Environment (build 1.8.0_422-422-b05)
OpenJDK 64-Bit Server VM (build 25.422-b05, mixed mode)



In [3]:
import os

print(os.getenv("JAVA_HOME"))
print(os.getenv("HADOOP_HOME"))

C:\Program Files\OpenLogic\jdk-8.0.422.05-hotspot
C:\hadoop-3.3.6


In [4]:
#Commentare se si importa il file in env anaconda
%pip install -r ../../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [5]:
import sparknlp

# create or get Spark Session
#spark = sparknlp.start()

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark NLP") \
    .master("local[*]") \
    .config("spark.driver.memory", "16G") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "2000M") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.4.1") \
    .getOrCreate()

In [7]:
# Verifica della versione di Spark NLP
print("Spark NLP version:", sparknlp.version())
print("Apache Spark version:", spark.version)

Spark NLP version: 5.4.1
Apache Spark version: 3.4.1


In [8]:
import sys
import os

# Aggiungi il percorso del modulo al PYTHONPATH
sys.path.append(os.path.abspath('../../Common'))

import UtilityNLP as nlpUtils
import UtilityClustering as cltUtils

c:\Users\pagan133\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [9]:
#Lettura Dataframe
df=spark.read.option("header",True).csv('../dati/input/esempio_frasi_1.csv')
df.cache()
df.show()

+--------------------+
|         descrizione|
+--------------------+
|ACCETTAZIONE DELL...|
|CONDIZIONI GENERA...|
| INFORMATIVA PRIVACY|
|ACCETTAZIONE DELL...|
|ACCETTAZIONE DELL...|
|ACCETTAZIONE DELL...|
|ACCETTAZIONE DELL...|
| INFORMATIVA PRIVACY|
| INFORMATIVA PRIVACY|
| INFORMATIVA PRIVACY|
| INFORMATIVA PRIVACY|
|      SCHEDA CLIENTE|
|      SCHEDA CLIENTE|
|ALLEGATO AMMINIST...|
|ALLEGATO AMMINIST...|
|ALLEGATO AMMINIST...|
|manca la sede leg...|
|      SCHEDA CLIENTE|
|      SCHEDA CLIENTE|
|      SCHEDA CLIENTE|
+--------------------+
only showing top 20 rows



In [10]:
#Definizione della pipeline per il Sentence Embedding con BERT
model = nlpUtils.nlp_pipeline_bert_sentence_embedding("descrizione").fit(df)
result_bert = model.transform(df)

sentence_detector_dl download started this may take some time.
Approximate size to download 514,9 KB
[ | ]
An error occurred while calling z:com.johnsnowlabs.nlp.pretrained.PythonResourceDownloader.downloadModel.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.FileSystem$4.<init>(FileSystem.java:2180)
	at org.apache.hadoop.fs.FileSystem.listLocatedStatus(FileSystem.java:2179)
	at org.apache.hadoop.fs.ChecksumFileSy

Py4JJavaError: An error occurred while calling z:com.johnsnowlabs.nlp.pretrained.PythonResourceDownloader.downloadModel.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.FileSystem$4.<init>(FileSystem.java:2180)
	at org.apache.hadoop.fs.FileSystem.listLocatedStatus(FileSystem.java:2179)
	at org.apache.hadoop.fs.ChecksumFileSystem.listLocatedStatus(ChecksumFileSystem.java:783)
	at org.apache.hadoop.mapred.

In [ ]:
#Caching della Sentence Embedding
result_bert.cache()
result_bert.display()

In [ ]:
print("# dataset originale: {}".format(df.count()))
print("# dataset nuovo: {}".format(result_bert.count()))

In [ ]:
#Converte gli embedding delle sentences in colonne separate nel DataFrame 
# (ogni vettore N-dimensionale viene tramutato in un dataframe di N colonne)
result_df_exp=nlpUtils.convert_sentence_embedding_in_col(result_bert,["idcase","descrizione","dataapertura"])
result_df_exp.display()

In [ ]:
#Applichiamo la PCA sulle features ottenute dal sentence embedding (explode dell'embedding)
col_features=result_df_exp.columns[4:]
result_df_exp_filled = result_df_exp.dropna()
result, pca_model, loadings=cltUtils.pipelineStandardPCA(result_df_exp_filled, col_features, 30)

In [ ]:
#Plottiamo la varianza mantenuta in base al numero di PCA da scegliere
cltUtils.cumulativePCwithVariance(pca_model)

In [ ]:
#Plot della Silhuette per decidere il corretto valore K di cluster
cltUtils.silhouetteClusteringKMeans(result,"pca_features",m=2,n=20,i=2)

In [ ]:
#Applicazione del clustering
predictions_cluster_final, final_model=cltUtils.defineClustering(result, 12)

In [ ]:
predictions_cluster_final.select("idcase","descrizione","dataapertura","sentence","prediction").distinct().display()

In [ ]:
# Plot in 3D interattivo delle 3 PC con varianza maggiore (PC1 - PC2 - PC3) 
cltUtils.plotPCA3DInterattivo(predictions_cluster_final.where("dataapertura >= '2024-06-01'"), 
                              features='pca_features', predictions='prediction', additional_column='descrizione')

In [ ]:
# Plot in 3D interattivo delle 3 PC con varianza maggiore (PC1 - PC2 - PC3) marcate per cluster associato
# I punti dello spazio associati allo stesso cluster hanno lo stesso colore associato 
cltUtils.plotClustering3DInterattivo(predictions_cluster_final.where("dataapertura >= '2024-06-01'"), 
                                     features='pca_features', predictions='prediction', additional_column='descrizione')

In [ ]:
#Applicazione della TF-IDF per ogni cluster:
# Restituisce le prime N parole con il peso TF-IDF più alto per ciascuna predizione
docs_per_topic = predictions_cluster_final.groupby('prediction').agg(concat_ws(' ', collect_list(col("sentence"))).alias('Doc'))

topN=nlpUtils.top_n_words(docs_per_topic, inputCol="Doc", outputCol="features", ngram=3, N=10, targetCol="prediction")

topN.display()

In [ ]:
docs_per_topic = predictions_cluster_final.groupby('prediction').agg(concat_ws(' ', collect_list(col("sentence"))).alias('Doc'))

topN_4gram=nlpUtils.top_n_words(docs_per_topic, inputCol="Doc", outputCol="features", ngram=4, N=10, targetCol="prediction")

topN_4gram.display()

In [ ]:
predictions_cluster_final.groupBy("prediction").count().display()

In [ ]:
#Creazione del dataset finale con frase in linguaggio naturale e cluster associato
predictions_cluster_final.select("idcase",
    "descrizione","prediction").join(topN,"prediction","left").display()